In [1]:
import mysql.connector
import pandas as pd

In [7]:
def open_database():
    con = mysql.connector.connect(user='dbadmin', password='washywashy',host='127.0.0.1',database='tests')
    return con, con.cursor()

def close_database(con, cursor):
    cursor.close()
    con.close()
    
def read_csv(filename):
    df = pd.read_csv(filename)
    return df

def select_columns(df, columns):
    return df[columns]


def init_tables():
    con, cursor = open_database()

    cursor.execute("drop table if exists tests.movies")
    cursor.execute("drop table if exists tests.genres")
   
    ## create GENRE table
    create_table_sql = """
        create table tests.genres (
            id int not null auto_increment,
            genre varchar(128),
            primary key(id)
        )
    """
    cursor.execute(create_table_sql)
    
    ## create MOVIE table
    create_table_sql = """
        create table tests.movies (
            id int not null auto_increment,
            film varchar(256),
            genre_id int,
            year int,
            profitability double,
            primary key(id),
            foreign key (genre_id) references genres(id) on delete cascade
        )
    """    
    cursor.execute(create_table_sql) 
    con.commit()
    
def get_data():
    df = read_csv('./data/movies.csv')
    df = select_columns(df, ['Film', 'Genre', 'Year', 'Profitability'])
    return df

def fill_genres():
    data = list()
    for d in data:
        data.append((d,))

    con, cursor = open_database()
    cursor.executemany("insert into tests.genres (genre) values (%s)", l)
    con.commit()
    close_database(con, cursor)
    
def get_genres():
    con, cursor = open_database()
    cursor.execute("select * from tests.genres")
    rows = cursor.fetchall()
    lookup = {}
    for r in rows:
        lookup[r[1]] = r[0]
    close_database(con, cursor)
    return lookup

def fill_movies():
    ## add a new genre_id column to df
    lookup = get_genres()
    df = get_data()
    genre_ids = []
    for g in df['Genre']:
        genre_ids.append(lookup[g])
    df['genre_id'] = genre_ids

    ## restructure df as we need in insert
    df = df[['Film', 'genre_id', 'Year', 'Profitability']]

    insert_sql = "insert into tests.movies (film, genre_id, year, profitability) values (%s, %s, %s, %s)"
    #data = tuple([tuple(x) for x in df.values])
    data = df.values.tolist()
    con, cursor = open_database()
    cursor.executemany(insert_sql, data)
    con.commit()
    close_database(con, cursor)
    
def show_movies_detail():
    con, cursor = open_database()
    cursor.execute("""
        select m.film, s.genre, m.year from tests.movies m
        join tests.genres s on m.genre_id = s.id
        order by m.year desc
    """)
    rows = cursor.fetchall()
    for r in rows:
        print(r)
    close_database(con, cursor)

In [8]:
# init_tables()
# fill_genres()
# fill_movies()
show_movies_detail()

('Jane Eyre', 'Romance', 2011)
('My Week with Marilyn', 'Drama', 2011)
('Midnight in Paris', 'Romence', 2011)
('Water For Elephants', 'Drama', 2011)
('Gnomeo and Juliet', 'Animation', 2011)
('Monte Carlo', 'Romance', 2011)
('Beginners', 'Comedy', 2011)
("New Year's Eve", 'Romance', 2011)
('Gnomeo and Juliet', 'Animation', 2011)
('Something Borrowed', 'Romance', 2011)
('One Day', 'Romance', 2011)
('Waiting For Forever', 'Romance', 2011)
('Twilight: Breaking Dawn', 'Romance', 2011)
('A Dangerous Method', 'Drama', 2011)
('Tangled', 'Animation', 2010)
('Our Family Wedding', 'Comedy', 2010)
('Remember Me', 'Drama', 2010)
('Love & Other Drugs', 'Comedy', 2010)
('Life as We Know It', 'Comedy', 2010)
('Letters to Juliet', 'Comedy', 2010)
('Leap Year', 'Comedy', 2010)
('Just Wright', 'Comedy', 2010)
('I Love You Phillip Morris', 'Comedy', 2010)
('Sex and the City 2', 'Comedy', 2010)
('Going the Distance', 'Comedy', 2010)
('Sex and the City Two', 'Comedy', 2010)
("She's Out of My League", 'Comed